In [1]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format

model = "DWT_catboost"
part = '/normal'
csv_files = [
    f"./timeseries/mestrado/resultados/{model}{part}/ANP_MONTHLY.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_HOURLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_WEEKLY_DATASET.csv",
    # f"./timeseries/mestrado/resultados/{model}{part}/PEDESTRIAN_COUNTS_DATASET.csv",
    # f"./timeseries/mestrado/resultados/{model}{part}/US_BIRTHS_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/ETTh1.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/ETTh2.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/ETTm1.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/ETTm2.csv",
]

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep=";")
        print(f"{csv_file.split('/')[-1]}: {len(df)} entries")
    except Exception as e:
        print(f"Erro ao ler {csv_file}: {e}")

ANP_MONTHLY.csv: 5642 entries
Erro ao ler ./timeseries/mestrado/resultados/DWT_catboost/normal/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv: [Errno 2] No such file or directory: './timeseries/mestrado/resultados/DWT_catboost/normal/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv'
Erro ao ler ./timeseries/mestrado/resultados/DWT_catboost/normal/M4_HOURLY_DATASET.csv: [Errno 2] No such file or directory: './timeseries/mestrado/resultados/DWT_catboost/normal/M4_HOURLY_DATASET.csv'
Erro ao ler ./timeseries/mestrado/resultados/DWT_catboost/normal/M4_WEEKLY_DATASET.csv: [Errno 2] No such file or directory: './timeseries/mestrado/resultados/DWT_catboost/normal/M4_WEEKLY_DATASET.csv'
Erro ao ler ./timeseries/mestrado/resultados/DWT_catboost/normal/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv: [Errno 2] No such file or directory: './timeseries/mestrado/resultados/DWT_catboost/normal/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv'
Erro ao ler ./timeseries/mestrado/resultados/DWT_catboost/normal/NN5_WEEKLY_

In [7]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/catboost/normal/M4_HOURLY_DATASET.csv",
    sep=";",
    )

df_agent[df_agent['dataset_index'] == 0]


,dataset_index,horizon,regressor,mape,pocid,smape,rmse,msmape,mae,test,predictions,start_test,final_test
0,0,48,catboost_normal,0.0520,95.7447,0.0498,50.9475,0.0498,36.1896,"[619.0, 565.0, 532.0, 495.0, 481.0, 467.0, 473...",[630.15559853 573.21928858 529.48314001 498.36...,2015-07-30 16:00:00,2015-08-01 15:00:00
8,0,48,catboost_normal,0.0772,93.6170,0.0817,59.3014,0.0817,50.6220,"[664.0, 550.0, 544.0, 505.0, 483.0, 469.0, 466...",[662.04055879 578.0305773 527.01390442 481.01...,2015-07-28 16:00:00,2015-07-30 15:00:00
15,0,48,catboost_normal,0.0600,87.2340,0.0624,60.3221,0.0623,44.0808,"[622.0, 558.0, 513.0, 476.0, 449.0, 437.0, 422...",[631.06387725 572.18403956 516.36529877 479.78...,2015-07-26 16:00:00,2015-07-28 15:00:00
20,0,48,catboost_normal,0.0583,85.1064,0.0557,39.5394,0.0557,32.7369,"[598.0, 547.0, 503.0, 474.0, 459.0, 450.0, 450...",[603.18721866 541.36052344 517.14223334 497.14...,2015-07-24 16:00:00,2015-07-26 15:00:00


In [5]:
ind = 0
df_viewer = df_agent[df_agent['dataset_index'] == ind]
df_viewer['description'].iloc[0]

'Models combined: statistical: ARIMA, catboost: DWT_catboost, rf: rf, svr: FT_svr, naive: NaiveMovingAverage'

In [ ]:
import pandas as pd

df_a = pd.read_csv(
    "./timeseries/mestrado/resultados/simple_selective_agent_qwen3=14b/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_a = df_a[df_a["final_test"] == "2024-11-30"]
df_a = df_a[df_a["dataset_index"] == ind]
df_a["description"].iloc[0]

In [8]:
df_agent = pd.read_csv(
    "./timeseries/mestrado/final/hybrid_agent/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

183
mape     76576144139471280.0000
pocid                   67.4615
smape                    1.4365
rmse                 69239.4483
msmape                   1.4365
mae                  68951.0944
dtype: float64


In [9]:
df_agent = pd.read_csv(
    "./timeseries/mestrado/final/BEST_CATEGORY_EACH_SERIE/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_agent = df_agent[df_agent['final_test'] == '2024-11-30']

print(len(df_agent))
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

182
mape     1867671054740872.7500
pocid                  62.8871
smape                   0.2305
rmse                 6529.2548
msmape                  0.2304
mae                  5715.5957
dtype: float64


In [ ]:
import re
import pandas as pd
import numpy as np


def extract_values(list_str):
    if isinstance(list_str, str):
        numbers = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", list_str)
        return [float(num) for num in numbers]
    return []


def get_best_model_by_rmse(models, dataset_index, final_test, val_test, base_path):
    """
    Encontra o melhor modelo baseado em RMSE para um dataset específico.

    Args:
        models: Lista de nomes dos modelos
        dataset_index: Índice do dataset
        final_test: Data do teste final
        base_path: Caminho base para os arquivos CSV

    Returns:
        Tupla (best_smape, best_model, best_predictions)
    """
    best_val_rmse = float("inf")
    best_val_model = ""
    best_predictions = []
    test = []
    for model in models:
        try:
            df = pd.read_csv(f"{base_path}/{model}/normal/ANP_MONTHLY.csv", sep=";")
            df = df[
                (df["final_test"] == val_test)
                & (df["dataset_index"] == dataset_index)
            ]

            if not df.empty:
                rmse = df.iloc[0]["smape"]
                if rmse < best_val_rmse:
                    # print(f"MODELO {best_val_model} é pior que {model}")
                    # print(f"antigo: {best_val_rmse} | novo: {rmse}")
                    best_val_rmse = rmse
                    best_val_model = model
                    # best_predictions = extract_values(df.iloc[0]["predictions"])
                    # test = extract_values(df.iloc[0]["test"])
        except Exception as e:
            print(f"Erro ao processar modelo {model}: {e}")

    df = pd.read_csv(f"{base_path}/{best_val_model}/normal/ANP_MONTHLY.csv", sep=";")
    df = df[(df["final_test"] == final_test) & (df["dataset_index"] == dataset_index)]
    best_predictions = extract_values(df.iloc[0]["predictions"])
    test = extract_values(df.iloc[0]["test"])

    return best_val_rmse, best_val_model, best_predictions, test


# Configurações
base_path = "/home/lucas/Documents/mestrado/Statistics_and_Seq2Seq/timeseries/mestrado/resultados"

# Definir todos os grupos de modelos
model_groups = {
    "statistical": ["ARIMA", "ETS", "THETA"],
    "catboost": [
        "catboost",
        "CWT_catboost",
        "DWT_catboost",
        "FT_catboost",
        "ONLY_CWT_catboost",
        "ONLY_DWT_catboost",
        "ONLY_FT_catboost",
    ],
    "rf": [
        "rf",
        "CWT_rf",
        "DWT_rf",
        "FT_rf",
        "ONLY_CWT_rf",
        "ONLY_DWT_rf",
        "ONLY_FT_rf",
    ],
    "svr": [
        "svr",
        "CWT_svr",
        "DWT_svr",
        "FT_svr",
        "ONLY_CWT_svr",
        "ONLY_DWT_svr",
        "ONLY_FT_svr",
    ],
    "naive": ["NaiveSeasonal", "NaiveMovingAverage"],
}

cols_serie = [
    "dataset_index",
    "horizon",
    "regressor",
    "mape",
    "pocid",
    "smape",
    "rmse",
    "msmape",
    "mae",
    "test",
    "predictions",
    "start_test",
    "final_test",
    "description",
]

# Processar todos os datasets
combined_results = []

import os
from all_functions import *
from sklearn.metrics import mean_absolute_percentage_error as mape

dataset = "ANP_MONTHLY"
exp_name = "SMAPE_BEST_CATEGORY_EACH_SERIE"
path_experiments = f"./timeseries/mestrado/resultados/{exp_name}/"
path_csv = f"{path_experiments}/{dataset}.csv"
final_test = "2024-11-30"
val_test = "2023-11-30"
# os.makedirs(path_experiments, exist_ok=True)
for i in range(0, 1):
    best_results = {}
    all_predictions = []

    # Encontrar o melhor modelo de cada grupo
    for group_name, models in model_groups.items():
        val_rmse, model, preds, test = get_best_model_by_rmse(
            models, i, final_test, val_test, base_path
        )
        print("------------------------")
        best_results[group_name] = {
            "val_rmse": val_rmse,
            "model": model,
            "predictions": preds,
        }

        # Coletar predições válidas para combinar
        if preds and len(preds) > 0:
            all_predictions.append(preds)

    # Combinar predições por média
    if all_predictions:
        # Converter para array numpy para facilitar a média
        predictions_array = np.array(all_predictions)
        combined_prediction = np.mean(predictions_array, axis=0).tolist()
    else:
        combined_prediction = []

    # Encontrar o melhor entre todos os grupos
    overall_best_group = min(best_results.items(), key=lambda x: x[1]["val_rmse"])

    print(f"\nDataset {i}:")
    for group_name, result in best_results.items():
        if result["model"]:
            print(
                f"  Melhor {group_name}: {result['model']} (RMSE na validação: {result['val_rmse']:.4f})"
            )

    preds_real = combined_prediction

    test = np.array(test)
    preds_real_array = np.array(preds_real)
    preds_real_reshaped = preds_real_array.reshape(1, -1)
    test_reshaped = test.reshape(1, -1)
    smape_result = calculate_smape(preds_real_reshaped, test_reshaped)
    # print(smape_result)
    rmse_result = calculate_rmse(preds_real_reshaped, test_reshaped)
    msmape_result = calculate_msmape(preds_real_reshaped, test_reshaped)
    # mase_result = calculate_mase(preds_real_reshaped, test_reshaped, training_set, seasonality)
    mae_result = calculate_mae(preds_real_reshaped, test_reshaped)
    mape_result = mape(test, preds_real_array)
    pocid_result = pocid(test, preds_real_array)

In [ ]:
import re
import pandas as pd
import numpy as np

def extract_values(list_str):
    if isinstance(list_str, str):
        numbers = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", list_str)
        return [float(num) for num in numbers]
    return []


def get_best_model_by_rmse(models, dataset_index, final_test, val_test, base_path):
    """
    Encontra o melhor modelo baseado em RMSE para um dataset específico.

    Args:
        models: Lista de nomes dos modelos
        dataset_index: Índice do dataset
        final_test: Data do teste final
        base_path: Caminho base para os arquivos CSV

    Returns:
        Tupla (best_smape, best_model, best_predictions)
    """
    best_val_rmse = float("inf")
    best_val_model = ""
    best_predictions = []
    test = []
    for model in models:
        try:
            df = pd.read_csv(f"{base_path}/{model}/normal/ANP_MONTHLY.csv", sep=";")
            df = df[
                (df["final_test"] == val_test) & (df["dataset_index"] == dataset_index)
            ]

            if not df.empty:
                rmse = df.iloc[0]["smape"]
                if rmse < best_val_rmse:
                    # print(f"MODELO {best_val_model} é pior que {model}")
                    # print(f"antigo: {best_val_rmse} | novo: {rmse}")
                    best_val_rmse = rmse
                    best_val_model = model
                    # best_predictions = extract_values(df.iloc[0]["predictions"])
                    # test = extract_values(df.iloc[0]["test"])
        except Exception as e:
            print(f"Erro ao processar modelo {model}: {e}")

    df = pd.read_csv(f"{base_path}/{best_val_model}/normal/ANP_MONTHLY.csv", sep=";")
    df = df[(df["final_test"] == final_test) & (df["dataset_index"] == dataset_index)]
    best_predictions = extract_values(df.iloc[0]["predictions"])
    test = extract_values(df.iloc[0]["test"])

    return best_val_rmse, best_val_model, best_predictions, test


# Configurações
base_path = "/home/lucas/Documents/mestrado/Statistics_and_Seq2Seq/timeseries/mestrado/resultados"
final_test = "2024-11-30"

# Definir todos os grupos de modelos
model_groups = {
    "statistical": ["ARIMA", "ETS", "THETA"],
    "catboost": ["catboost", "CWT_catboost", "DWT_catboost", "FT_catboost", 
                 "ONLY_CWT_catboost", "ONLY_DWT_catboost", "ONLY_FT_catboost"],
    "rf": ["rf", "CWT_rf", "DWT_rf", "FT_rf", "ONLY_CWT_rf", "ONLY_DWT_rf", "ONLY_FT_rf"],
    "svr": ["svr", "CWT_svr", "DWT_svr", "FT_svr", "ONLY_CWT_svr", "ONLY_DWT_svr", "ONLY_FT_svr"],
    "naive": ["NaiveSeasonal", "NaiveMovingAverage"]
}

cols_serie = [
    "dataset_index",
    "horizon",
    "regressor",
    "mape",
    "pocid",
    "smape",
    "rmse",
    "msmape",
    "mae",
    "test",
    "predictions",
    "start_test",
    "final_test",
    "description",
]

# Processar todos os datasets
combined_results = []

import os
from all_functions import *
from sklearn.metrics import mean_absolute_percentage_error as mape

dataset = "ANP_MONTHLY"
exp_name = "SMAPE_BEST_CATEGORY_EACH_SERIE"
path_experiments = (
        f"./timeseries/mestrado/resultados/{exp_name}/"
    )
path_csv = f"{path_experiments}/{dataset}.csv"
os.makedirs(path_experiments, exist_ok=True)
val_test = "2023-11-30"
for i in range(0, 182):
    best_results = {}
    all_predictions = []

    # Encontrar o melhor modelo de cada grupo
    for group_name, models in model_groups.items():
        val_rmse, model, preds, test = get_best_model_by_rmse(models, i, final_test,val_test, base_path)
        best_results[group_name] = {
            'val_rmse': val_rmse,
            'model': model,
            'predictions': preds
        }

        # Coletar predições válidas para combinar
        if preds and len(preds) > 0:
            all_predictions.append(preds)

    # Combinar predições por média
    if all_predictions:
        # Converter para array numpy para facilitar a média
        predictions_array = np.array(all_predictions)
        combined_prediction = np.mean(predictions_array, axis=0).tolist()
    else:
        combined_prediction = []

    # Encontrar o melhor entre todos os grupos
    overall_best_group = min(best_results.items(), key=lambda x: x[1]['val_rmse'])

    print(f"\nDataset {i}:")
    print(combined_prediction)
    # Mostrar os melhores de cada grupo
    for group_name, result in best_results.items():
        if result['model']:
            print(f"  Melhor {group_name}: {result['model']} (SMAPE: {result['val_rmse']:.4f})")
            
    
    preds_real = combined_prediction
    
    test = np.array(test)
    preds_real_array = np.array(preds_real)
    preds_real_reshaped = preds_real_array.reshape(1, -1)
    test_reshaped = test.reshape(1, -1)
    smape_result = calculate_smape(preds_real_reshaped, test_reshaped)
    # print(smape_result)
    rmse_result = calculate_rmse(preds_real_reshaped, test_reshaped)
    msmape_result = calculate_msmape(preds_real_reshaped, test_reshaped)
    # mase_result = calculate_mase(preds_real_reshaped, test_reshaped, training_set, seasonality)
    mae_result = calculate_mae(preds_real_reshaped, test_reshaped)
    mape_result = mape(test, preds_real_array)
    pocid_result = pocid(test, preds_real_array)
    
    description = "Models combined: " + ", ".join([f"{group}: {best_results[group]['model']}" for group in best_results if best_results[group]['model']])
    data_serie = {
            "dataset_index": f"{i}",
            "horizon": "12",
            "regressor": "BEST_CATEGORY_EACH_SERIE",
            "mape": mape_result,
            "pocid": pocid_result,
            "smape": smape_result,
            "rmse": rmse_result,
            "msmape": msmape_result,
            "mae": mae_result,
            "test": [test.tolist()],
            "predictions": [preds_real],
            "start_test": "INICIO",
            "final_test": final_test,
            "description": description,
            # 'training_time': times[0],
            # 'prediction_time': times[1],
        }

    if not os.path.exists(path_csv):
            pd.DataFrame(columns=cols_serie).to_csv(path_csv, sep=";", index=False)

    print("Salvando resultados...\n")
    df_new = pd.DataFrame(data_serie)
    df_new.to_csv(path_csv, sep=";", mode="a", header=False, index=False)


    # print(best_results)

In [15]:
from typing import Dict, List, Union, Tuple
import numpy as np
from sklearn.linear_model import LinearRegression


def _ensure_array(y: Union[float, List[float]], n: int) -> np.ndarray:
    """Garante que y tem tamanho n; se for escalar, repete para formar array de tamanho n."""
    if isinstance(y, (list, tuple, np.ndarray)):
        y_arr = np.asarray(y, dtype=float)
        if y_arr.shape[0] != n:
            raise ValueError(
                f"Lista y_val tem tamanho {y_arr.shape[0]} mas esperava {n}"
            )
        return y_arr
    else:
        return np.full(shape=(n,), fill_value=float(y), dtype=float)


def check_input_shapes(
    val_preds: Dict[str, List[float]], test_preds: Dict[str, List[float]]
) -> int:
    """
    Verifica consistência mínima:
    - todas as listas em val_preds têm o mesmo tamanho n_val
    - todos os modelos em val_preds existem em test_preds (test_preds pode ter listas de tamanho diferente)
    Retorna n_val.
    """
    if not val_preds:
        raise ValueError("val_preds está vazio")
    lengths = {k: len(v) for k, v in val_preds.items()}
    n_vals = set(lengths.values())
    if len(n_vals) != 1:
        raise ValueError(f"Tamanhos inconsistentes nas listas de validação: {lengths}")
    n_val = n_vals.pop()

    # checar test_preds contém as chaves
    for k in val_preds:
        if k not in test_preds:
            raise ValueError(
                f"Modelo '{k}' presente em val_preds mas ausente em test_preds"
            )
        if not isinstance(test_preds[k], (list, tuple, np.ndarray)):
            raise ValueError(f"test_preds['{k}'] precisa ser lista/np.array")
    return n_val


def shift_correction(
    val_preds: Dict[str, List[float]],
    y_val: List[float],
    test_preds: Dict[str, List[float]],
) -> Dict[str, List[float]]:
    """
    Corrige previsões adicionando um bias por modelo:
      bias = mean(y_val) - mean(pred_val_model)
    Aplica bias às previsões de teste e retorna dicionário de previsões corrigidas.
    """
    n_val = check_input_shapes(val_preds, test_preds)
    y_val_arr = _ensure_array(y_val, n_val)
    y_mean = float(np.mean(y_val_arr))

    corrected = {}
    for name, preds_val in val_preds.items():
        preds_val_arr = np.asarray(preds_val, dtype=float)
        bias = y_mean - float(np.mean(preds_val_arr))
        preds_test_arr = np.asarray(test_preds[name], dtype=float)
        corrected[name] = (preds_test_arr + bias).tolist()

    return corrected

import re

def extract_values(list_str):
    if isinstance(list_str, str):
        numbers = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", list_str)
        return [float(num) for num in numbers]
    return []

In [55]:
val_preds = {
    "ARIMA": [
        400.71678399,
        663.87332953,
        929.11779741,
        928.19414546,
        820.45108076,
        942.73181293,
        917.04772083,
        777.88168678,
        643.23803468,
        743.95702414,
        786.77982244,
        730.3459295,
    ],
    "FT_catboost": [
        400.71678399,
        663.87332953,
        929.11779741,
        928.19414546,
        820.45108076,
        942.73181293,
        917.04772083,
        777.88168678,
        643.23803468,
        743.95702414,
        786.77982244,
        730.3459295,
    ],
    "ONLY_DWT_rf": [
        355.62580122,
        450.92858957,
        417.68229707,
        1332.2369491,
        529.15589532,
        921.31116252,
        974.07278312,
        809.75277656,
        747.63653457,
        549.20459147,
        1694.72720717,
        1188.96124668,
    ],
    "FT_svr": [
        480.87089365,
        520.23308461,
        632.27912708,
        657.58230609,
        471.22153901,
        522.99862274,
        496.83943868,
        477.60744965,
        398.98449469,
        337.1191924,
        405.94991964,
        352.95463704,
    ],
    "NaiveSeasonal": [
        163.35,
        124.32,
        1767.006,
        1003.093,
        76.45,
        2023.998,
        2251.496,
        1055.33,
        245.87,
        162.73,
        670.983,
        -1.13686838e-13,
    ],
}
y_val = [
    47.51,
    0.0,
    62.15,
    4214.105,
    92.68,
    1030.353,
    2585.757,
    1267.991,
    0.0,
    2517.924,
    855.162,
    250.91,
]

test_preds = {
    "ARIMA": [
        626.54445435,
        867.25753704,
        1140.44178255,
        1218.40255972,
        922.16557401,
        1025.36568259,
        1277.64676113,
        1043.62573386,
        871.6622963,
        977.16178989,
        938.74572533,
        920.28328892,
    ],
    "FT_catboost": [
        287.72484798,
        -296.80893321,
        259.47301569,
        -221.27725565,
        255.15551412,
        254.38388207,
        203.31202369,
        793.22870178,
        1008.24206288,
        1084.23653574,
        554.45271123,
        564.5496957,
    ],
    "ONLY_DWT_rf": [
        1167.69032389,
        718.71243323,
        352.10670718,
        844.43235289,
        2291.78410799,
        889.53650482,
        993.63343095,
        1258.16041135,
        595.12303392,
        1212.79863338,
        1000.14051588,
        2113.93667613,
    ],
    "FT_svr": [
        -387.22267965,
        -1602.8661116,
        -1434.76910943,
        -1765.4953112,
        -561.58789508,
        353.94495089,
        816.7784578,
        971.05251116,
        649.41751243,
        1055.17371582,
        282.25494992,
        149.97195324,
    ],
    "NaiveSeasonal": [
        47.51,
        -2.27373675e-13,
        62.15,
        4214.105,
        92.68,
        1030.353,
        2585.757,
        1267.991,
        -2.27373675e-13,
        2517.924,
        855.162,
        250.91,
    ],
}
y_test = [2101.917,
 4194.392,
 4443.881,
 3153.49,
 1398.42,
 680.292,
 -3.036,
 2000.117,
 2160.356,
 1995.957,
 4128.637,
 4555.324]

In [56]:
from sklearn.metrics import mean_absolute_percentage_error as mape

new_preds = linear_correction(val_preds=val_preds, y_val=y_val, test_preds=test_preds)

preds_list = [preds for preds in new_preds.values()]
combined = np.mean(preds_list, axis=0)
mape(y_test, combined)

42.51127291866553

In [ ]:
import pandas as pd
index = 12
df_arima = pd.read_csv(
    "./timeseries/mestrado/resultados/NaiveSeasonal/normal/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_arima = df_arima[df_arima["final_test"] == "2020-11-30"]
df_arima = df_arima[df_arima["dataset_index"] == index]
extract_values(df_arima['test'].iloc[0])

[673.091,
 645.903,
 2587.536,
 1970.165,
 3338.866,
 642.657,
 328.47,
 290.66,
 729.046,
 654.321,
 1048.09,
 719.027]

In [67]:
import pandas as pd

index = 12
df_arima = pd.read_csv(
    "./timeseries/mestrado/resultados/ARIMA/normal/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
df_arima = df_arima[df_arima["final_test"] == "2024-11-30"]
df_arima = df_arima[df_arima["dataset_index"] == index]
extract_values(df_arima["predictions"].iloc[0])

[626.54445435,
 867.25753704,
 1140.44178255,
 1218.40255972,
 922.16557401,
 1025.36568259,
 1277.64676113,
 1043.62573386,
 871.6622963,
 977.16178989,
 938.74572533,
 920.28328892]